# GraphRAG Relevance, Factuality and Synthesis Evaluation

## Step 0: environment set up

In [8]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph
from libs import create_vector_index
import pandas as pd
from conn import connect2Googlesheet,retrieval_rel_docs, get_concatenate_df, apply_metric
from libs import context_builder, chunk_finder, enhanced_chunk_finder
# Force reload of the .env file
load_dotenv()

True

In [9]:
# Connect to Neo4j database
try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URL"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD")
    )
    print("Connected to Neo4j database successfully.")
except ValueError as e:
    print(f"Could not connect to Neo4j database: {e}")

Connected to Neo4j database successfully.


## Step 1: Create vector index

In [10]:
#create_vector_index(graph, "entities")

## Step 2: Load questions from google sheet

In [11]:
spreadsheet = connect2Googlesheet()

# Select the worksheet: relevance
worksheet = spreadsheet.get_worksheet(2)  

# Get all records as a list of dictionaries
data = worksheet.get_all_records()

# Convert to Pandas DataFrame
df_MedQ = pd.DataFrame(data)
df_MedQ.head()

,condition,number,docs,Question,Mahmud's Note,status,comments,
0,ARDS,1,ACURASYS,Does early administration of neuromuscular blocking agents increases the ventilator free days?,Like,,,
1,ARDS,2,ACURASYS,Do patients with severe ARDS being treated with neuromuscular blocking agents have increased muscle weakness?,Replace,fixed,,
2,ARDS,3,ROSE,"In patients with moderate to severe ARDS, does early use of continuous neuromuscular blockade improve mortality?","Maybe this question: In patients with moderate to severe ARDS, does early use of continuous neuromuscular blockade improve mortality?",fixed,,
3,ARDS,4,ROSE,Do patients with moderate-to-severe ARDS have a significance difference in mortality rate beween patients who recieved an early and continous cisatracurium infusion than those with usual care approach with lighter sedation targets?,Local question (not sure if this is the aim of your project) It will be nice as second step after proving the general summarization is working but focusing in general summarization would be priority in my opinion so you can have meanigful tool.,fixed,Wrong concept since PEEP by itself is mandatory component in ventilator.,"Does the use of neuromuscular blockers in patients with moderate-to-severe ARDS impact cardiovascular stability, particularly in terms of vasopressor requirements and hemodynamic effects, compared to sedation strategy without routine neuromuscular blockade?"
4,ARDS,5,FACTT,"Among patients with ALI/ARDS, does a conservative fluid management strategy improves lung function and decrease ventilator days compared to liberal strategy?","Local question (not sure if this is the aim of your project) consider (WikiJournal): In patients with ALI/ARDS that are intubated and receiving positive pressure ventilation, how does the conservative compare to the liberal fluid management strategy in reducing mortality?",fixed,Check if studies defined conservative by CVP < 4 or elese just dont mention how much the CVP (i prefer the last approach),


## Step 3: Relevance check for top K questions

In [12]:
def retrieval_rel_docs (graph, questions, top_k=5):
    top_k_questions = questions.head(top_k)
    # Initialize a list to store the results
    results = []
    # Iterate over the top k questions
    for index, row in top_k_questions.iterrows():
        question_number = index + 1  # Assuming the question number is the index + 1
        question = row['Question']  # Replace 'Question' with the actual column name for questions in df_MedQ
        
        # Generate response for the question
        # context = context_builder(graph, question, method="vector")
        filenames , output = enhanced_chunk_finder(graph, question)
        # Extract relevant documents from the response content
        # docs = response.choices[0].message.content  # Adjust this based on the actual response structure
        
        # Append the result to the list
        results.append({'Question number': question_number, 'Question': question, 'Retrieved FileName': filenames, 'Chunks': output})

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results, columns=['Question number', 'Question', 'Retrieved FileName', 'Chunks'])
    
    return results_df
#retrieval_rel_docs (graph, questions, top_k=1)

In [13]:
# Set pandas display options to show the full text content
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
topk = 5
results_df = retrieval_rel_docs(graph, df_MedQ, top_k=topk)
results_df
# results_df.to_csv('./outputs/retrieved_docs_results.csv', index=False)

Question number  \
0                1   
1                2   
2                3   
3                4   
4                5   

                                                                                                                                                                                                                                  Question  \
0                                                                                                                                           Does early administration of neuromuscular blocking agents increases the ventilator free days?   
1                                                                                                                            Do patients with severe ARDS being treated with neuromuscular blocking agents have increased muscle weakness?   
2                                                                                                                         In patients with moderate to severe ARDS, does early use of continuous neuromuscular blockade improve mortality?   
3  Do patients with moderate-to-severe ARDS have a significance difference in mortality rate beween patients who recieved an early and continous cisatracurium infusion than those with usual care approach with lighter sedation targets?   
4                                                                            Among patients with ALI/ARDS, does a conservative fluid management strategy improves lung function and decrease ventilator days compared to liberal strategy?   

                                                                                      Retrieved FileName  \
0                                                           [ACURASYS.pdf, PESDCOMV.pdf, FMWSCPARDS.pdf]   
1                                                                                 [LSPA.pdf, ETSDMV.pdf]   
2                                                                       [ROSE.pdf, ENB.pdf, NBSARDS.pdf]   
3                                               [LSPA.pdf, ROSE.pdf, TOF-ARDS.pdf, ENB.pdf, NBSARDS.pdf]   
4  [FACTT.pdf, FMWSCPARDS.pdf, ROSE.pdf, OSCILLATE.pdf, ACURASYS.pdf, ETSDMV.pdf, TOF-ARDS.pdf, ENB.pdf]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [14]:
# old results
old_results = pd.read_csv('./outputs/relevant_docs.csv')
old_results

,Unnamed: 0,Question,Docs
0,0,Relevace Q1,"ROSE, ACURASYS"
1,1,Relevace Q2,"ROSE, ACURASYS"
2,2,Relevace Q3,ROSE
3,3,Relevace Q4,ROSE
4,4,Relevace Q5,FACTT
5,5,Relevace Q6,FACTT
6,6,Relevace Q7,ARDSNet
7,7,Relevace Q8,ARDSNet
8,8,Relevace Q9,PROSEVA
9,9,Relevace Q10,OSCILLATE
